In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem

from pymoo.factory import get_crossover, get_mutation, get_sampling
from pymoo.algorithms.soo.nonconvex.ga import GA

from pooling_functions import calc_pool_conc, MinConcProblem, MinNumCntrProblem

Load data:

In [2]:
datay = pd.read_csv('data/yearly_damages.csv', index_col=0)
datay.head()

,ASM,KHM,COK,FJI,PYF,IDN,KIR,LAO,MYS,MHL,...,MWI,MUS,MOZ,SOM,TZA,ZMB,ZWE,BWA,ZAF,SWZ
0,0.000000e+00,1.174686e+05,61567.633482,0.000000,2066.621255,4.932370e+09,0.0,1.468178e+05,1.363927e+10,0.000000,...,0.0,4.132596e+08,1.240871e+07,1.518563e+07,8.905124e+09,0.0,0.0,0.0,1.297125e+06,0.0
1,0.000000e+00,2.135547e+07,0.000000,0.000000,1817.411519,0.000000e+00,0.0,2.456694e+06,5.648761e+06,81.644624,...,0.0,3.394774e+05,1.486315e+08,0.000000e+00,7.669974e+06,0.0,0.0,0.0,0.000000e+00,0.0
2,2.615973e+07,6.706448e+08,2374.730234,0.000000,42.486543,6.417800e+07,0.0,0.000000e+00,3.386322e+05,0.000000,...,0.0,6.093447e+07,1.342145e+07,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
3,6.236010e+07,3.473528e+07,702363.195433,4825.253640,0.000000,0.000000e+00,0.0,0.000000e+00,2.667397e+09,0.000000,...,0.0,1.005025e+05,1.356082e+04,8.197128e+05,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
4,0.000000e+00,2.488902e+07,0.000000,158.037936,0.000000,3.155947e+09,0.0,0.000000e+00,0.000000e+00,0.000000,...,0.0,5.339540e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0


Load damages by region:

In [3]:
reg_names = ['EAP', 'CAC', 'SA', 'SEA']

df_reg = {}

for reg_name in reg_names:
    regs = np.loadtxt('data/preprocessing/{}.txt'.format(reg_name), dtype=str)

    df_reg.update({reg_name: datay[regs]})

Define alpha:

In [4]:
RT = 200

alpha = 1-1/RT
alpha

0.995

Find optimal pool for each region:

In [10]:
for reg in df_reg.keys():

    bools = df_reg[reg] >= np.quantile(df_reg[reg], alpha, axis=0)
    problem = MinConcProblem(df_reg[reg], bools.values, alpha, calc_pool_conc)

    print("FIND OPTIMAL POOL FOR REGION {}".format(reg))
    print("Step 1 - Pool with the minimum concentration")

    algorithm = GA(pop_size=df_reg[reg].shape[1],
    sampling=get_sampling("bin_random"),
    crossover=get_crossover("bin_hux"),
    mutation=get_mutation("bin_bitflip"),
    eliminate_duplicates=True)

    res_reg = minimize(problem,
                         algorithm,
                         ('n_gen', 750),
                         verbose=False,
                         save_history=True,
                         pf=np.array(0.0))

    print("Step 2 - Smallest set of countries within this pool maintaining min conc\n")

    cntries = res_reg.X.astype(int)

    reg_datay_opt = df_reg[reg][df_reg[reg].columns[cntries > 0]]
    bools_opt = reg_datay_opt >= np.quantile(reg_datay_opt, alpha, axis=0)

    problem = MinNumCntrProblem(reg_datay_opt, bools_opt.values, alpha, calc_pool_conc, res_reg.F[0])

    algorithm = GA(pop_size=reg_datay_opt.shape[1],
    sampling=get_sampling("bin_random"),
    crossover=get_crossover("bin_hux"),
    mutation=get_mutation("bin_bitflip"),
    eliminate_duplicates=True)

    res_reg2 = minimize(problem,
                         algorithm,
                         ('n_gen', 750),
                         verbose=False,
                         save_history=True,
                         pf=np.array(0.0))

FIND OPTIMAL POOL FOR REGION EAP
Step 1 - Pool with the minimum concentration
Step 2 - Smallest set of countries within this pool maintaining min conc

FIND OPTIMAL POOL FOR REGION CAC
Step 1 - Pool with the minimum concentration
Step 2 - Smallest set of countries within this pool maintaining min conc

FIND OPTIMAL POOL FOR REGION SA
Step 1 - Pool with the minimum concentration
Step 2 - Smallest set of countries within this pool maintaining min conc

FIND OPTIMAL POOL FOR REGION SEA
Step 1 - Pool with the minimum concentration
Step 2 - Smallest set of countries within this pool maintaining min conc

